In [1]:
input_dirs = [r"C:\Users\Egor\VsCode project\lcd_display\data\new_big_aug_datasets\train\images", r"C:\Users\Egor\VsCode project\lcd_display\data\new_big_aug_datasets\valid\images", r"C:\Users\Egor\VsCode project\lcd_display\data\new_big_aug_datasets\test\images"]
output_dirs = [r"C:\Users\Egor\VsCode project\lcd_display\data\new_big_aug_rotate_180\train\images", r"C:\Users\Egor\VsCode project\lcd_display\data\new_big_aug_rotate_180\val\images", r"C:\Users\Egor\VsCode project\lcd_display\data\new_big_aug_rotate_180\test\images"]

labels = ['train','val', 'test']
dfs = {
    'train': [],
    'val': [],
    'test': []
}

In [ ]:
python model/test_resnet.py --model weight/best_lcd_resnet183.pth --images_dir data/test/images --csv data/test/test.csv

In [ ]:
docker run -d --name vpr_notebook_gpu_2 --gpus all -p 32712:8888 -v /mnt/external_usb_hdd/6YL/Datasets/SberRobotics/maps:/home/docker_mmpr/Datasets:rw -v /home/pinkin_ek/reranking_test:/workspace:rw egor_p:cuda126 jupyter lab --ip=0.0.0.0 --allow-root --ServerApp.token='vpr_egor'

In [2]:
from PIL import Image, ImageOps
import pandas as pd
import os

image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')


for input_dir, output_dir, split in zip(input_dirs, output_dirs, labels):
    os.makedirs(output_dir, exist_ok=True)

    # Сначала берем только изображения и сортируем — так порядок будет предсказуем
    all_files = [f for f in os.listdir(input_dir) if f.lower().endswith(image_extensions)]
    all_files.sort()

    # Перебираем только изображения и используем enumerate по этому списку
    for idx, filename in enumerate(all_files):
        input_path = os.path.join(input_dir, filename)
        try:
            img = Image.open(input_path)

            # Учитываем EXIF-ориентацию (чтобы корректно видеть результат везде)
            img = ImageOps.exif_transpose(img)

            flip = 0
            # Если хотим переворачивать каждое второе изображение, считаем по idx из filtered list
            if idx % 2 == 0:   # или == 1 — в зависимости с какой позиции начинать
                img = img.rotate(180)   # или img = img.transpose(Image.ROTATE_180)
                flip = 1

            dfs[split].append({
                'filename': filename,
                'flip': flip
            })

            out_path = os.path.join(output_dir, filename)
            img.save(out_path)
            # Для отладки можно раскомментировать:
            # print(idx, filename, 'flipped' if flip else 'not flipped')

        except Exception as e:
            print(f"Ошибка при обработке {filename}: {e}")

df_train = pd.DataFrame(dfs['train'])
df_val = pd.DataFrame(dfs['val'])
df_test = pd.DataFrame(dfs['test'])

# Сохраняем в CSV
df_train.to_csv('train.csv', index=False, encoding='utf-8')
df_val.to_csv('val.csv', index=False, encoding='utf-8')
df_test.to_csv('test.csv', index=False, encoding='utf-8')

print("\n" + "="*50)
print("ИТОГИ:")
print(f"train.csv: {len(df_train)} записей")
print(f"  Перевернуто (flip=1): {df_train['flip'].sum()}")
print(f"  Не перевернуто (flip=0): {len(df_train) - df_train['flip'].sum()}")
print(f"val.csv: {len(df_val)} записей")
print(f"  Перевернуто (flip=1): {df_val['flip'].sum()}")
print(f"  Не перевернуто (flip=0): {len(df_val) - df_val['flip'].sum()}")
print(f"test.csv: {len(df_test)} записей")
print(f"  Перевернуто (flip=1): {df_test['flip'].sum()}")
print(f"  Не перевернуто (flip=0): {len(df_test) - df_test['flip'].sum()}")
print("="*50)


ИТОГИ:
train.csv: 5499 записей
  Перевернуто (flip=1): 2750
  Не перевернуто (flip=0): 2749
val.csv: 508 записей
  Перевернуто (flip=1): 254
  Не перевернуто (flip=0): 254
test.csv: 256 записей
  Перевернуто (flip=1): 128
  Не перевернуто (flip=0): 128


In [4]:
# запустите в той же сессии, где у вас model
from ultralytics import YOLO

model = YOLO(r"C:\Users\Egor\VsCode project\lcd_display\weight\epoch5_verb.pt")
results = model.val(data=r"C:\Users\Egor\VsCode project\lcd_display\data\gleb_lcd_roboflow\data.yaml", split='test')

print(f"mAP50: {results.box.map50}")
print(f"mAP50-95: {results.box.map}")
print(f"Precision: {results.box.mp}")
print(f"Recall: {results.box.mr}")



Ultralytics 8.3.235  Python-3.12.5 torch-2.9.1+cpu CPU (AMD Ryzen 5 5600H with Radeon Graphics)
YOLO11l summary (fused): 190 layers, 25,289,335 parameters, 0 gradients, 86.6 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 55.420.1 MB/s, size: 2.7 KB)
val: Scanning C:\Users\Egor\VsCode project\lcd_display\data\test\labels.cache... 0 images, 65 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 65/65 32.3Kit/s 0.0s
WARNING Labels are missing or empty in C:\Users\Egor\VsCode project\lcd_display\data\test\labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 5/5 7.4s/it 36.9s<12.1s


c:\Users\Egor\VsCode project\lcd_display\venv\Lib\site-packages\ultralytics\utils\metrics.py:653: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
c:\Users\Egor\VsCode project\lcd_display\venv\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\Egor\VsCode project\lcd_display\venv\Lib\site-packages\ultralytics\utils\metrics.py:697: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
c:\Users\Egor\VsCode project\lcd_display\venv\Lib\site-packages\numpy\_core\_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
c:\Users\Egor\VsCode project\lcd_display\venv\Lib\site-packages\ultralytics\utils\metrics.py:697: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
c:\Users\Egor\VsCode project\lcd_display\venv\Lib\site-packages\numpy\_c

                   all         65          0          0          0          0          0
WARNING no labels found in detect set, can not compute metrics without labels
Speed: 1.2ms preprocess, 562.7ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to C:\Users\Egor\VsCode project\runs\detect\val3
mAP50: 0.0
mAP50-95: 0.0
Precision: 0.0
Recall: 0.0
